In [1]:
from typing import cast

import pandas as pd
import simdjson
from pydantic import BaseModel
from rich import print

from xdf_types import XDFData

In [2]:
class AnswerRound(BaseModel):
    round_number: int
    latency_applied: int  # Actual latency in milliseconds from LatencyMarkers stream
    delays_experienced: int  # Answer to "Did you experience delays..."
    task_difficulty: int  # Answer to "How difficult was it..."
    felt_controlling: int  # Answer to "I felt like I was controlling..."
    felt_part_of_body: int  # Answer to "It felt like the robot was part of my body"


class Participant(BaseModel):
    submission_id: int
    created: str
    participant_number: int
    gender: str
    age: int
    dominant_hand: str  # Can be "Right hand", "Left hand", "Ambidextrous", etc.
    robotics_experience: int
    answer_time_ms: float
    rounds: list[AnswerRound]


ignored_participants = [4]

In [3]:
questionnaire_df = pd.read_csv(
    "data/questionnaire_data-561422-2025-11-11-1622.csv", sep=";"
)

In [4]:
# Static columns (demographics)
static_cols = [
    "$submission_id",
    "$created",
    "Participant number",
    "What is your gender",
    "How old are you?",
    "What is your dominant hand?",
    "How experienced are you with robotic systems?",
    "$answer_time_ms",
]

# The repeating question columns
repeating_questions = [
    "Did you experience delays between your actions and the robot&#39;s movements?",
    "How difficult was it to perform the task?",
    "I felt like I was controlling the movement of the robot",
    "It felt like the robot was part of my body",
]

# Count how many rounds we have
all_cols = questionnaire_df.columns.tolist()
# Remove static columns to get only repeating ones
repeating_cols = [col for col in all_cols if col not in static_cols]
num_rounds = len(repeating_cols) // len(repeating_questions)

print(f"Number of rounds: {num_rounds}")

Number of rounds: 10

In [5]:
participants: list[Participant] = []

for _, row in questionnaire_df.iterrows():
    participant_number = int(row["Participant number"])
    if participant_number in ignored_participants:
        continue

    data_file = f"./data/sub-{participant_number:03d}/sub-{participant_number:03d}_ses-_task-_run-001.json"
    json_parser = simdjson.Parser()
    data: XDFData = cast(XDFData, json_parser.load(data_file))

    # Extract latency markers from the data
    latency_markers_stream = next(
        (
            stream
            for stream in data["streams"]
            if stream["info"]["name"] == "LatencyMarkers"
        ),
        None,
    )

    if latency_markers_stream is None:
        raise ValueError(
            f"LatencyMarkers stream not found for participant {participant_number}"
        )

    # Extract latencies from condition_advance markers
    # Format: ["condition_advance|rep_1|200ms|condition_1"]
    latencies_by_round = []
    for marker in latency_markers_stream["time_series"]:
        marker_str = marker[0]
        if marker_str.startswith("condition_advance|"):
            parts = marker_str.split("|")
            if len(parts) >= 3:
                latency_str = parts[2]  # e.g., "200ms"
                latency_ms = int(latency_str.replace("ms", ""))
                latencies_by_round.append(latency_ms)

    # Parse answer rounds
    rounds = []
    for round_idx in range(num_rounds):
        # Column names have suffixes like .1, .2, etc. (pandas duplicate column naming)
        # First occurrence has no suffix, then .1, .2, .3, etc.
        suffix = f".{round_idx}" if round_idx > 0 else ""

        round_data = AnswerRound(
            round_number=round_idx + 1,
            latency_applied=latencies_by_round[round_idx]
            if round_idx < len(latencies_by_round)
            else 0,
            delays_experienced=int(row[f"{repeating_questions[0]}{suffix}"]),
            task_difficulty=int(row[f"{repeating_questions[1]}{suffix}"]),
            felt_controlling=int(row[f"{repeating_questions[2]}{suffix}"]),
            felt_part_of_body=int(row[f"{repeating_questions[3]}{suffix}"]),
        )
        rounds.append(round_data)

    # Create participant with all rounds
    participant = Participant(
        submission_id=int(row["$submission_id"]),
        created=str(row["$created"]),
        participant_number=participant_number,
        gender=str(row["What is your gender"]),
        age=int(row["How old are you?"]),
        dominant_hand=str(row["What is your dominant hand?"]),
        robotics_experience=int(row["How experienced are you with robotic systems?"]),
        answer_time_ms=float(row["$answer_time_ms"]),
        rounds=rounds,
    )
    participants.append(participant)

In [6]:
print(participants)

[
    Participant(
        submission_id=38973009,
        created='2025-11-05',
        participant_number=1,
        gender='Male',
        age=24,
        dominant_hand='Left hand',
        robotics_experience=1,
        answer_time_ms=3801195.0,
        rounds=[
            AnswerRound(
                round_number=1,
                latency_applied=200,
                delays_experienced=2,
                task_difficulty=2,
                felt_controlling=4,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=2,
                latency_applied=50,
                delays_experienced=2,
                task_difficulty=1,
                felt_controlling=4,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=3,
                latency_applied=150,
                delays_experienced=2,
                task_difficulty=2,
                felt_controlling=4,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=4,
                latency_applied=100,
                delays_experienced=2,
                task_difficulty=2,
                felt_controlling=4,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=5,
                latency_applied=0,
                delays_experienced=2,
                task_difficulty=4,
                felt_controlling=3,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=6,
                latency_applied=150,
                delays_experienced=3,
                task_difficulty=3,
                felt_controlling=3,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=7,
                latency_applied=200,
                delays_experienced=3,
                task_difficulty=3,
                felt_controlling=3,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=8,
                latency_applied=50,
                delays_experienced=2,
                task_difficulty=2,
                felt_controlling=3,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=9,
                latency_applied=100,
                delays_experienced=2,
                task_difficulty=3,
                felt_controlling=2,
                felt_part_of_body=1
            ),
            AnswerRound(
                round_number=10,
                latency_applied=0,
                delays_experienced=3,
                task_difficulty=3,
                felt_controlling=3,
                felt_part_of_body=1
            )
        ]
    ),
    Participant(
        submission_id=38974447,
        created='2025-11-05',
        participant_number=2,
        gender='Female',
        age=22,
        dominant_hand='Right hand',
        robotics_experience=1,
        answer_time_ms=2302137.0,
        rounds=[
            AnswerRound(
                round_number=1,
                latency_applied=200,
                delays_experienced=2,
                task_difficulty=4,
                felt_controlling=4,
                felt_part_of_body=3
            ),
            AnswerRound(
                round_number=2,
                latency_applied=100,
                delays_experienced=2,
                task_difficulty=3,
                felt_controlling=5,
                felt_part_of_body=3
            ),
            AnswerRound(
                round_number=3,
                latency_applied=150,
                delays_experienced=2,
                task_difficulty=2,
                felt_controlling=5,
                felt_part_of_body=4
            ),
            AnswerRound(
                round_number=4,
                latency_applied=0,
                delays_experienced=1,
                task_d